In [ ]:
pip install ipynb

In [ ]:
#!pip install ipynb

In [ ]:
import pandas as pd
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
from datetime import date
import glob, os
from joblib import dump, load
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
import time
from ipynb.fs.full.Data_Preparation import getProcessedData
from ipynb.fs.full.Data_Preparation import getFilenames
import random
random.seed(42)

In [ ]:
path = os.getcwd()

In [ ]:
# Define the list of currency tickers used in the analysis
fx_tic = ['USDEUR',
            'USDJPY',
            'USDGBP',
            'USDCHF',
            'USDNZD',
            'USDCAD',
            'USDSEK',
            'USDDKK',
            'USDNOK',
            'EURJPY',
            'EURGBP',
            'EURCHF',
            'EURNZD',
            'EURCAD',
            'EURSEK',
            'EURDKK',
            'EURNOK']
# Create a dictionary that maps each currency ticker to an index
fx_dic = {'USDEUR': 0,
 'USDJPY': 1,
 'USDGBP': 2,
 'USDCHF': 3,
 'USDNZD': 4,
 'USDCAD': 5,
 'USDSEK': 6,
 'USDDKK': 7,
 'USDNOK': 8,
 'EURJPY': 9,
 'EURGBP': 10,
 'EURCHF': 11,
 'EURNZD': 12,
 'EURCAD': 13,
 'EURSEK': 14,
 'EURDKK': 15,
 'EURNOK': 16}

In [ ]:
# Create a dictionary that defines the top 20 features for each currency ticker. These features are selected by SFFS
feat_20={
    'USDEUR': [0, 6, 11, 12, 16, 22, 23, 24, 25, 28, 30, 31, 33, 38, 44, 45, 48, 49, 54, 59],
    'USDJPY': [0, 13, 14, 15, 16, 18, 19, 20, 21, 22, 26, 28, 31, 33, 35, 38, 41, 47, 51, 53],
    'USDGBP': [1, 2, 5, 13, 16, 18, 23, 24, 25, 31, 35, 36, 38, 41, 42, 46, 48, 51, 52, 54],
    'USDCHF': [2, 5, 7, 8, 10, 11, 17, 25, 27, 35, 41, 43, 47, 48, 51, 52, 53, 54, 55, 57],
    'USDNZD': [5, 6, 8, 10, 13, 14, 17, 20, 21, 23, 24, 25, 33, 37, 40, 43, 51, 52, 53, 57],
    'USDCAD': [5, 8, 10, 11, 17, 18, 20, 23, 24, 25, 31, 33, 35, 40, 42, 52, 54, 55, 56, 57],
    'USDSEK': [7, 8, 10, 11, 17, 18, 21, 23, 24, 25, 26, 27, 31, 33, 35, 40, 42, 43, 53, 55],
    'USDDKK': [1, 3, 7, 13, 14, 17, 22, 23, 26, 27, 28, 29, 30, 31, 35, 44, 48, 51, 52, 53],
    'USDNOK': [0, 2, 4, 5, 7, 15, 17, 18, 20, 22, 23, 26, 27, 33, 34, 37, 47, 49, 51, 57],
    'EURJPY': [0, 1, 4, 9, 13, 20, 21, 23, 24, 25, 26, 29, 31, 41, 44, 47, 51, 53, 54, 56],
    'EURGBP': [4, 5, 7, 8, 13, 17, 18, 20, 21, 23, 24, 25, 27, 41, 42, 47, 51, 52, 54, 55],
    'EURCHF': [4, 11, 12, 14, 17, 18, 19, 26, 29, 32, 33, 34, 39, 40, 45, 47, 49, 50, 51, 52],
    'EURNZD': [8, 9, 10, 11, 16, 18, 21, 23, 26, 28, 31, 35, 36, 38, 46, 50, 51, 52, 53, 55],
    'EURCAD': [5, 8, 10, 11, 18, 20, 21, 23, 24, 25, 26, 28, 32, 33, 41, 42, 43, 46, 48, 58],
    'EURSEK': [2, 4, 8, 10, 11, 13, 15, 20, 22, 23, 33, 39, 40, 41, 44, 48, 51, 54, 55, 59],
    'EURDKK': [1, 6, 8, 10, 17, 21, 26, 30, 31, 35, 37, 43, 44, 45, 47, 48, 50, 51, 55, 58],
    'EURNOK': [2, 8, 10, 11, 13, 17, 18, 21, 23, 24, 25, 40, 41, 42, 43, 47, 53, 56, 57, 58]
}

In [ ]:
# Define a function to compute the XGBoost feature importance for each ticker
def xgb_feature_importance(ticker_names):
    # List to store the XGB feature importance for each ticker
    xgbfi_gain_result = []
    # Dictionary to store the top 5 important features for each ticker
    xgbfi_gain_result_top_5 = {}

    # Loop through each ticker
    for i in range(len(ticker_names)):
        stock_name = ticker_names[i]

        # Data split
        X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(fx_tic)[fx_dic[stock_name]])
        # For each market, we only use top 20 features selected by SFFS
        X_train = X_train.iloc[:,feat_20[stock_name]]
        X_val1 = X_val1.iloc[:,feat_20[stock_name]]
        X_val2 = X_val2.iloc[:,feat_20[stock_name]]
        X_test = X_test.iloc[:,feat_20[stock_name]]

        # Merge the training sets and validation sets to form a complete training set
        X_train_total = pd.concat([X_train, X_val1, X_val2])
        y_train_total = pd.concat([y_train, y_val1, y_val2])

        # Load the XGBoost model with the previously found best hyperparameters
        xgb_model = load(path + '/model_weights/' + stock_name +'xgb.h5')
        # Retrain the XGBoost model parameters on the newly defined training set
        xgb_model.fit(X_train_total, y_train_total)
        
        # Extract XGB feature importance from the trained XGBoost model 
        # Calculated by the average gain across all splits the feature is used in
        result = xgb_model.get_booster().get_score(importance_type='gain')

        # Sort the results by feature importance
        ord = dict(sorted(result.items(), key=lambda item: item[1],reverse=True))

        # Append the sorted feature importance results to the list
        xgbfi_gain_result.append(ord)

        # Extract and save the top 5 features from the results
        lst = list(ord)

        feat_5 = lst[:5]

        xgbfi_gain_result_top_5[stock_name] = feat_5

    return xgbfi_gain_result, xgbfi_gain_result_top_5

In [ ]:
start_time = time.time()
# Calculate and save the XGBoost feature importances
xgbfi_gain_result, xgbfi_gain_result_top_5 = xgb_feature_importance(fx_tic)
end_time = time.time()
elapsed_time = end_time - start_time

[21:24:39] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[21:24:39] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[21:24:39] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [ ]:
print(f"The time cost of XGB feature importance is:{elapsed_time:.2f}s")

The time cost of XGB feature importance is:2.07s


In [ ]:
xgbfi_gain_result

[{'KC_LOWER': 6.0096211433410645,
  'TRIMA': 4.998025417327881,
  'DO_UP': 4.80155086517334,
  'NATR': 4.74209451675415,
  'Typical_Price': 4.680840969085693,
  'CHANDELIER_LONG': 4.671468734741211,
  'AROONOSC': 4.350463390350342,
  'MIDPRICE': 4.267056941986084,
  'EMA': 4.069373607635498,
  'PSL': 3.961669921875,
  'HMA': 3.9536056518554688,
  'DEMA': 3.912022829055786,
  'VORTEX_NEG': 3.901817560195923,
  'AROON_DOWN': 3.8481712341308594,
  'BB_MAVG': 3.6544651985168457,
  'ROC': 3.610898017883301,
  'KC_UPPER': 3.4542858600616455,
  'Open': 3.1399641036987305,
  'AROON_UP': 3.0964763164520264},
 {'WMA': 9.798383712768555,
  'TRIMA': 7.855625152587891,
  'CHANDELIER_SHORT': 7.728633403778076,
  'BB_HB': 7.308779716491699,
  'BB_MAVG': 7.036061763763428,
  'ADX': 6.858535289764404,
  'BB_LB': 6.339840888977051,
  'MOM': 6.230974197387695,
  'ULCERINDEX': 6.03523063659668,
  'PSL': 5.976057529449463,
  'RVGI': 5.685225963592529,
  'MI': 5.294023036956787,
  'DX': 5.048496246337891,
 

In [ ]:
xgbfi_gain_result_top_5

{'USDEUR': ['KC_LOWER', 'TRIMA', 'DO_UP', 'NATR', 'Typical_Price'],
 'USDJPY': ['WMA', 'TRIMA', 'CHANDELIER_SHORT', 'BB_HB', 'BB_MAVG'],
 'USDGBP': ['High', 'NATR', 'BBWIDTH', 'VORTEX_NEG', 'AROON_UP'],
 'USDCHF': ['Low', 'CMO', 'CHANDELIER_LONG', 'RVGI', 'TRIX'],
 'USDNZD': ['MOM', 'BB_HB', 'SO', 'WI', 'AROONOSC'],
 'USDCAD': ['CHOP', 'MSD', 'AROONOSC', 'VORTEX_NEG', 'ROC'],
 'USDSEK': ['VORTEX_POS', 'ADX', 'ATR', 'PERCENT_B', 'NATR'],
 'USDDKK': ['MIDPRICE', 'DEMA', 'CHANDELIER_LONG', 'RVI', 'DPO'],
 'USDNOK': ['BB_LB', 'TRIMA', 'Low', 'MI', 'BIAS'],
 'EURJPY': ['KC_UPPER', 'NATR', 'CCI', 'TRIX', 'AROON_DOWN'],
 'EURGBP': ['PERCENT_B', 'DX', 'MI', 'CHANDELIER_SHORT', 'MSD'],
 'EURCHF': ['KAMA', 'MI', 'KC_LOWER', 'ULCERINDEX', 'BIAS'],
 'EURNZD': ['MSD', 'BB_MAVG', 'BBWIDTH', 'KAMA', 'SO'],
 'EURCAD': ['ATR', 'ULCERINDEX', 'CHANDELIER_LONG', 'SO', 'ROC'],
 'EURSEK': ['KC_UPPER', 'ACC_UP', 'VORTEX_POS', 'CCI', 'BB_LB'],
 'EURDKK': ['ATR', 'NATR', 'SO', 'High', 'Typical_Price'],
 'EURNO

In [ ]:
# Save the complete feature importance results to a h5 file
dump(xgbfi_gain_result, path + '/XGB_FI_result/xgbfi_gain_result_new.h5')

['/content/gdrive/MyDrive/Capstone-main/XGB_FI_result_New/xgbfi_gain_result_new.h5']

In [ ]:
# Save the top 5 features for each ticker to a h5 file
dump(xgbfi_gain_result_top_5, path + '/XGB_FI_result/xgbfi_gain_result_top_5_new.h5')

['/content/gdrive/MyDrive/Capstone-main/XGB_FI_result_New/xgbfi_gain_result_top_5_new.h5']